# Label and Feature Creation

In this notebook, I will import the single-column dataframe of appellate opinions and create columns with labels and features. 

In [98]:
import io
import re
import pandas as pd
import pickle
import operator

In [99]:
# Open the dataframe
infile = open('ProjectData/df_clean.data', 'rb')
df = pickle.load(infile)
infile.close()

In [100]:
df.reset_index(inplace=True, drop=True)

In [101]:
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3922 entries, 0 to 3921
Data columns (total 1 columns):
Opinion    3922 non-null object
dtypes: object(1)
memory usage: 30.8+ KB


(None,                                              Opinion
 0   an unpublished opinion of the north carolina ...
 1  no. coa11-246 north carolina court of appeals ...
 2  no. coa08-347 north carolina court of appeals ...
 3  michael harrison gregory and wife, vivian greg...
 4  atlantic contracting and material company, inc...)

### 1. Creating a New Column for the File Numbers
This is more experimental than functional.

In [102]:
# capture file number into new column
def coa(string_):
    try:
        pat_coa_number = re.search("no.? ?coa.? ?(\d{2}-\d{1,5})",string_)
        return pat_coa_number.group(1)
    except:
        return('00-000')

In [103]:
coa(df.Opinion[3921])

'20-112'

In [104]:
coa_numbers = []
for i in range(len(df.Opinion)):
    x = coa(df.Opinion[i])
    coa_numbers.append(x)

In [105]:
placeholder=pd.Series(coa_numbers)
df["File_Numbers"] = placeholder.values

In [106]:
df.head(10)

,Opinion,File_Numbers
0,an unpublished opinion of the north carolina ...,19-563
1,no. coa11-246 north carolina court of appeals ...,11-246
2,no. coa08-347 north carolina court of appeals ...,08-347
3,"michael harrison gregory and wife, vivian greg...",05-885
4,"atlantic contracting and material company, inc...",02-1087
5,an unpublished opinion of the north carolina c...,13-222
6,in the court of appeals of north carolina no....,17-112
7,in the court of appeals of north carolina no....,15-862
8,no. coa11-1447 north carolina court of appeal...,11-1447
9,an unpublished opinion of the north carolina c...,13-248


### 2. Creating the Labels (Affirmed, Reversed, etc.)

The labels were created using the regex patterns below. The ultimate fuction below was created over many iterations. Initially, there were approximately 300 errors; the model was tweaked to reduce errors while maintaining reliability. Ultimately, 25 rows were dropped as errors because the cases were not beneficial to the model (i.e., they did not include a relevant summary judgment decision, the opinion on summary judgment was entwined with other components, etc.); 454 "affirmed-in-part" rows were dropped since they're not the binary outcome needed; and 408 dismissals were dropped since they don't have a usable outcome.  

In [108]:
# 
def labels(string_):
    """
    This function will extract the outcome from a given string (opinion).
    Each of the 'try' statements extracts the labels with decreasing 
    degrees of confidence (first by the one-word sentence, then within
    10 words of the statement regarding concurrence, typically near the 
    end of the opinion; then clipping the last 150 chars. of the opinion 
    and looking for the associated label-words, and finally looking for 
    label-words within 5-10 words of the often-occurring phrase, 'for the
    reasons set forth above').
    """
    try:
        try:
            try:
                try:  #this level has the highest confidence of getting an accurate label, based upon review of opinions (a single-word sentence)
                    labels = re.search("\.. ?(affirmed?)\.|\.?(reversed?)\.|(affirmed in part)|\.?(dismissed)\.",string_)
                    x = labels.group(1)
                    y = labels.group(2)
                    z = labels.group(3)
                    w = labels.group(4)
                    not_none = [x,y,z]
                    a = [i for i in not_none if i != None]
                    return a[0]
                except:  # slightly less confidence; looks for outcome word within 10 words of "concur", which frequently is at the end of a unanymous opinion
                    labels = re.search("(?:concurs?\W+(?:\w+\W+){0,40}?((affirmed in part)|reversed|affirmed|dismissed|no error|vacated)|((affirmed in part)|affirmed|reversed|dismissed|no error|vacated)\W+(?:\w+\W+){0,40}?concurs?)", string_)
                    #print("Group 0:", labels.group(0), "\nGroup 1:", labels.group(1), "\nGroup 2:", labels.group(2), "\nGroup 3:", labels.group(3), "\nGroup 4:", labels.group(4))
                    x = labels.group(1)
                    y = labels.group(2)
                    z = labels.group(3)
                    w = labels.group(4)
                    not_none = [x,y,z,w]
                    a = [i for i in not_none if i != None]
                    #print("This is resulting list a:", a)
                    return a[0]
            except: #slightly less confidence; if both of the previous methods fail, this clips the last 150 chars of the opinion for any of the outcome words
                clip = string_[-150:]
    #             print(clip)
                labels2 = re.search("('affirmed in part'|reversed|affirmed|dismissed|'affirm in part'|affirm|reverse|dismiss|improvidently allowed)",clip)
                return labels2.group(0)
        except: 
            labels = re.search("(?:reasons set forth?\W+(?:\w+\W+){0,5}?((affirm in part)|reverse|affirm|dismiss|no error|vacated?)|((affirm in part)|affirm|reversed?|dismiss|no error|vacated?)\W+(?:\w+\W+){0,10}?reasons set forth?)", string_)
            #print("Group 0:", labels.group(0), "\nGroup 1:", labels.group(1), "\nGroup 2:", labels.group(2), "\nGroup 3:", labels.group(3), "\nGroup 4:", labels.group(4))
            x = labels.group(1)
            y = labels.group(2)
            z = labels.group(3)
            w = labels.group(4)
            not_none = [x,y,z,w]
            a = [i for i in not_none if i != None]
            #print("This is resulting list a:", a)
            return a[0]
    except:
        return('error')

In [109]:
# Test Cell 
labels(df.Opinion[2092])

'affirmed'

In [110]:
# Apply labels to the DataFrame
labels_list = []
for i in range(len(df.Opinion)):
    x = labels(df.Opinion[i])
    labels_list.append(x)
    
labels_series = pd.Series(labels_list)
df["Result"] = labels_series.values

In [111]:
df.Result.value_counts()

affirmed                 2070
reversed                  691
affirmed in part          454
dismissed                 392
reverse                   122
no error                   70
vacated                    64
error                      25
affirm                     18
improvidently allowed      14
dismiss                     2
Name: Result, dtype: int64

In [112]:
df['Result'].replace(['reverse','affirm', 'dismiss','no error', 'vacated', 'improvidently allowed'],
                     ['reversed','affirmed','dismissed', 'affirmed', 'reversed', 'dismissed'], inplace=True)

# The model will treat 'no error' as 'affirmed' and 'vacated' as 'reversed'

In [113]:
df.Result.value_counts()

affirmed            2158
reversed             877
affirmed in part     454
dismissed            408
error                 25
Name: Result, dtype: int64

In [114]:
# Drop rows with 'error', 'dismissed', and 'affirmed in part' -- see section header, above
drop_list1 = df.loc[df['Result'] == 'error']
drop_list2 = df.loc[df['Result'] == 'affirmed in part']
drop_list3 = df.loc[df['Result'] == 'dismissed']

In [115]:
drop_list = list(drop_list1.index) + list(drop_list2.index) + list(drop_list3.index)

In [116]:
df.drop(drop_list, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [117]:
df.Result.value_counts()

affirmed    2158
reversed     877
Name: Result, dtype: int64

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3035 entries, 0 to 3034
Data columns (total 3 columns):
Opinion         3035 non-null object
File_Numbers    3035 non-null object
Result          3035 non-null object
dtypes: object(3)
memory usage: 71.3+ KB


### 3. Create Case-Type Feature By Sorting With Keywords

I created a simple sorting function which takes a dictionary of case types with associated keywords, and then it generates a popularity count of the various keywords, returning the highest-ranking case type for a given opinion. The dictionary was revised over many iterations and reviews; for instance, some words were not unique, were misleading, or needed leading/trailing spaces. 

In [119]:
# This dictionary contains types of law with associated, typically unique keywords
case_type_dict = {'premises':['premises', 'attractive nuisance', 'dangerous condition', 'slip and fall',
                            'defective condition', 'dog bite'], 
                  'car_crash':['collision', 'vehicle', 'motorist'], 
                  'med_mal':['medical malpractice','health care profession', 'same or similar community', 
                             'rule 9(j)', 'rule 702(b)'], 
                  'contract':['formation', 'recission', 'specific performance', 'incidental damages',
                              'consequential damages', 'statute of frauds'], 
                  'family_law':['divorce', 'custody', 'maintenance', 'child support', 'separation agreement',
                               'prenuptual', 'postnuptual'], 
                  'estates':['intestate', 'probate', 'revocable trust', 'irrevocable trust', 'testator',
                             'holographic', 'residue'], 
                  'landlord_tenant':['lease', 'landlord', 'security deposit', ' rent ', 'chapter 42'], 
                  'construction':['building defect', 'water intrusion', 'construction defect'], 
                  'property':['easement', 'fee simple', 'tenants in common', 'joint tenants', 'nuisance', 
                             'eminent domain', 'escheat', 'replevin', 'zoning', 'mortgage', 'foreclosure'], 
                  'unfair_deceptive':['unfair and deceptive', 'chapter 75'],
                  'defamation':['libel', 'slander', 'defamatory', 'defamation'],
                  'governmental':['sovereign immunity', 'official capacity'],
                  'discrimination':['wrongful discharge', 'discrimination', 'retaliation', 'retaliatory',
                                    'retaliatory employment discrimination act', 'discriminatory'],
                  'wrongful_death':['wrongful death']}

# 'dram_shop':['dram shop'] -- removed, only 2 that were not superseded by "car crash"

In [120]:
def case_type_sorter(dict_of_keywords, string):
    """ 
    This function takes a dictionary of case types and associated 
    keywords, assigns points for the frequency of the keywords
    of a given case type, and returns the case type with the highest
    number of points, as well as a confidence measure.  The dict_of_keywords 
    should be a dictionary of case-type keys and keyword values; the string 
    should be a single string.
    """
    counter_dict = {}

    # Iterate through dictionary, counting frequency of each keyword in the string/Opinion
    for key, values in dict_of_keywords.items():
        counter_dict[key] = 0
        for value in values:
            count = string.count(value)
            existing_count = counter_dict[key]
            counter_dict[key] = count + existing_count
    
    # Get total points for all keywords
    values = counter_dict.values()
    total_count = sum(values)
    
    likely_case_type = max(counter_dict.items(), key=operator.itemgetter(1))[0]
    try:
        confidence = str(round((counter_dict[likely_case_type]/total_count)*100,2))+'%'
    except:
        confidence = 'n/a'
    
    return likely_case_type, confidence

In [1]:
# Test cell
case_type_sorter(case_type_dict, df.Opinion[2032])
# 1276 - Dram shop
# 1407 - Dram shop
# 1576 - Car crash 89%
# 1605 - Car crash 36%
# 1763 - Car crash 63%
# 2032 - Car crash 76%

NameError: name 'case_type_sorter' is not defined

In [122]:
# Apply case_type and confidence level columns to the DataFrame

case_type_list = []
case_type_confidence = []
for i in range(len(df.Opinion)):
    y,z = case_type_sorter(case_type_dict, df.Opinion[i])
    case_type_list.append(y)
    case_type_confidence.append(z)

case_type_series = pd.Series(case_type_list)
case_confidence_series = pd.Series(case_type_confidence)
df["Case_Type"] = case_type_series.values
df["Case_Type_Confidence"] = case_confidence_series.values

In [123]:
df.sample(20)

,Opinion,File_Numbers,Result,Case_Type,Case_Type_Confidence
1752,no. coa11-1537 north carolina court of appeal...,11-1537,affirmed,estates,100.0%
36,an unpublished opinion of the north carolina c...,10-257,affirmed,unfair_deceptive,45.45%
2637,an unpublished opinion of the north carolina ...,17-1267,affirmed,contract,100.0%
1839,no. coa02-188 north carolina court of appeals ...,02-188,affirmed,unfair_deceptive,100.0%
520,"carl d. buckland, sr., and northfield developm...",99-1347,reversed,family_law,75.0%
2138,in the supreme court of north carolina no. 365...,00-000,reversed,premises,n/a
2431,"george c. jones, jr., petitioner, v. robert j....",98-1023,reversed,property,100.0%
2735,no. coa12-160 north carolina court of appeals ...,12-160,affirmed,car_crash,96.77%
1415,an unpublished opinion of the north carolina ...,17-870,affirmed,landlord_tenant,83.33%
278,an unpublished opinion of the north carolina c...,09-1119,affirmed,contract,60.0%


# need to drop cases "reverse ... denial" -- maybe as a separate action 

In [125]:
string2 = "this is a string that says rent different rentyish fajrent"
count = string2.count(" rent")
count

2

In [126]:
df.Case_Type.value_counts()

premises            524
contract            455
property            401
car_crash           387
landlord_tenant     355
family_law          264
governmental        154
unfair_deceptive    137
med_mal              94
discrimination       91
estates              79
defamation           50
wrongful_death       32
construction         10
dram_shop             2
Name: Case_Type, dtype: int64

# DO THE BELOW CELL FOR CONSTRUCTION (SEEMS LOW) AND REVIEW ~20 FOR EACH

In [135]:
total = 0
for i in range(len(df)):
    if df.Opinion[i].count('dram shop') >= 1:
        print(df.index[i])
        
# Why are dram shop so low if there are only 6 in the DF -- or does it really matter?

1276
1407
1576
1605
1763
2032


## 4. Check Cases With a Reversal of a Denial

## 5.  Extract Feature: Trial-Court Judge

## 6. Extract Feature: County